In [1]:
import pandas as pd
import numpy as np
import torch
from torch.nn.functional import softplus

In [2]:
### load the data and process it
away_data = pd.read_csv('https://raw.githubusercontent.com/metrica-sports/sample-data/master/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Away_Team.csv', skiprows=2)
home_data = pd.read_csv('https://raw.githubusercontent.com/metrica-sports/sample-data/master/data/Sample_Game_1/Sample_Game_1_RawTrackingData_Home_Team.csv', skiprows=2)


In [121]:
away_data.head()

,Period,Frame,Time [s],Player25,Unnamed: 4,Player15,Unnamed: 6,Player16,Unnamed: 8,Player17,...,Player24,Unnamed: 24,Player26,Unnamed: 26,Player27,Unnamed: 28,Player28,Unnamed: 30,Ball,Unnamed: 32
0,1,1,0.04,0.90509,0.47462,0.58393,0.20794,0.67658,0.46710,0.67310,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.45472,0.38709
1,1,2,0.08,0.90494,0.47462,0.58393,0.20794,0.67658,0.46710,0.67310,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.49645,0.40656
2,1,3,0.12,0.90434,0.47463,0.58393,0.20794,0.67658,0.46710,0.67310,...,0.37833,0.27383,NaN,NaN,NaN,NaN,NaN,NaN,0.53716,0.42556
3,1,4,0.16,0.90377,0.47463,0.58351,0.20868,0.67640,0.46762,0.67279,...,0.37756,0.27473,NaN,NaN,NaN,NaN,NaN,NaN,0.55346,0.42231
4,1,5,0.20,0.90324,0.47464,0.58291,0.21039,0.67599,0.46769,0.67253,...,0.37663,0.27543,NaN,NaN,NaN,NaN,NaN,NaN,0.55512,0.40570


In [135]:
device = 'cpu'; dtype = torch.float32


jitter = 1e-12 ## prevents division by zero when player is stationary

## set nans to large negative value -- this makes pitch control for players who aren't involved negligibly small everywhere
# on the pitch
home_pos = np.array([np.asarray(home_data.iloc[:,range(3 + j*2,3 + j*2 +2)]) for j in range(14)]) * np.array([105,68])
np.nan_to_num(home_pos,copy=False,nan=-1000)
away_pos = np.array([np.asarray(away_data.iloc[:,range(3 + j*2,3 + j*2 +2)]) for j in range(14)]) * np.array([105,68])
np.nan_to_num(away_pos,copy=False,nan=-1000)
ball_pos = (np.asarray(home_data.iloc[:,range(31,33)]) * np.array([105,68]))[:,None,None,:]


# defining the delta in seconds between frames
tt = np.asarray(home_data['Time [s]'])
dt = tt[1:] - tt[:-1] 
# velocity in both x and y axis + jitter 
# jitter prevents division by zero when player is stationary
home_v = (home_pos[:,1:,:] - home_pos[:,:-1,:])/dt[:,None] + jitter
np.nan_to_num(home_v,copy=False,nan=-1000)
away_v = (away_pos[:,1:,:] - away_pos[:,:-1,:])/dt[:,None] + jitter
np.nan_to_num(away_v,copy=False,nan=-1000)
# velocity -1000 not really necessary?

home_pos.shape

(14, 145006, 2)

In [138]:

# what is the point of this block
home_pos = home_pos[:,1:,None,None,:]
away_pos = away_pos[:,1:,None,None,:]
home_v = home_v[:,:,None,None,:]
away_v = away_v[:,:,None,None,:]
ball_pos = ball_pos[None,1:]

ball_pos.shape

(1, 0, 0, 145005, 1, 1, 2)

In [152]:
n_grid_points_x = 50
n_grid_points_y = 30
# create grid based on tensors
XX,YY = torch.meshgrid(torch.linspace(0,105,n_grid_points_x, device = device, dtype=dtype),
                       torch.linspace(0,68,n_grid_points_y,device=device,dtype=dtype))

XX

tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  2.1429,   2.1429,   2.1429,  ...,   2.1429,   2.1429,   2.1429],
        [  4.2857,   4.2857,   4.2857,  ...,   4.2857,   4.2857,   4.2857],
        ...,
        [100.7143, 100.7143, 100.7143,  ..., 100.7143, 100.7143, 100.7143],
        [102.8571, 102.8571, 102.8571,  ..., 102.8571, 102.8571, 102.8571],
        [105.0000, 105.0000, 105.0000,  ..., 105.0000, 105.0000, 105.0000]])

In [143]:
ti,wi = np.polynomial.legendre.leggauss(50) ## used for numerical integration later on
ti = torch.tensor(ti,device = device,dtype=dtype)
wi = torch.tensor(wi,device=device,dtype=dtype)
ti.shape

torch.Size([50])

In [145]:
target_position = torch.stack([XX,YY],2)[None,None,:,:,:] # all possible positions
target_position

tensor([[[[[  0.0000,   0.0000],
           [  0.0000,   2.3448],
           [  0.0000,   4.6897],
           ...,
           [  0.0000,  63.3103],
           [  0.0000,  65.6552],
           [  0.0000,  68.0000]],

          [[  2.1429,   0.0000],
           [  2.1429,   2.3448],
           [  2.1429,   4.6897],
           ...,
           [  2.1429,  63.3103],
           [  2.1429,  65.6552],
           [  2.1429,  68.0000]],

          [[  4.2857,   0.0000],
           [  4.2857,   2.3448],
           [  4.2857,   4.6897],
           ...,
           [  4.2857,  63.3103],
           [  4.2857,  65.6552],
           [  4.2857,  68.0000]],

          ...,

          [[100.7143,   0.0000],
           [100.7143,   2.3448],
           [100.7143,   4.6897],
           ...,
           [100.7143,  63.3103],
           [100.7143,  65.6552],
           [100.7143,  68.0000]],

          [[102.8571,   0.0000],
           [102.8571,   2.3448],
           [102.8571,   4.6897],
           ...,
     

In [105]:
device = 'cpu'; dtype = torch.float32


jitter = 1e-12 ## prevents division by zero when player is stationary

## set nans to large negative value -- this makes pitch control for players who aren't involved negligibly small everywhere
# on the pitch
home_pos = np.array([np.asarray(home_data.iloc[:,range(3 + j*2,3 + j*2 +2)]) for j in range(14)]) * np.array([105,68])
np.nan_to_num(home_pos,copy=False,nan=-1000)
away_pos = np.array([np.asarray(away_data.iloc[:,range(3 + j*2,3 + j*2 +2)]) for j in range(14)]) * np.array([105,68])
np.nan_to_num(away_pos,copy=False,nan=-1000)
ball_pos = (np.asarray(home_data.iloc[:,range(31,33)]) * np.array([105,68]))[:,None,None,:]


# defining the delta in seconds between frames
tt = np.asarray(home_data['Time [s]'])
dt = tt[1:] - tt[:-1] 
# velocity in both x and y axis + jitter 
# jitter prevents division by zero when player is stationary
home_v = (home_pos[:,1:,:] - home_pos[:,:-1,:])/dt[:,None] + jitter
np.nan_to_num(home_v,copy=False,nan=-1000)
away_v = (away_pos[:,1:,:] - away_pos[:,:-1,:])/dt[:,None] + jitter
np.nan_to_num(away_v,copy=False,nan=-1000)
# velocity -1000 not really necessary?




# severe changes in the shape of the arrays
# I am not 100% how and why this is necessary
# This is probably necessary for later 
home_pos = home_pos[:,1:,None,None,:]
away_pos = away_pos[:,1:,None,None,:]
home_v = home_v[:,:,None,None,:]
away_v = away_v[:,:,None,None,:]
ball_pos = ball_pos[None,1:]


## set up evaluation grid and set some pitch control parameters (these are taken from the FoT code)
reaction_time = 0.7
max_player_speed = 5.
average_ball_speed = 15.
sigma = np.pi / np.sqrt(3.) / 0.45
lamb = 4.3
n_grid_points_x = 50
n_grid_points_y = 30
# create grid based on tensors
XX,YY = torch.meshgrid(torch.linspace(0,105,n_grid_points_x, device = device, dtype=dtype),
                       torch.linspace(0,68,n_grid_points_y,device=device,dtype=dtype))
target_position = torch.stack([XX,YY],2)[None,None,:,:,:] # all possible positions


# the weights and the x-points of the Gauss–Legendre quadrature are set up and stored as torch tensors
# I assume that, as the Gauss–Legendre quadrature spans the range of [-1, 1] (length of 2), the 50 points selected
# represent the number of frames in a 2 second time period based on a framerate of 25Hz (i.e. 25 frames / second)
ti,wi = np.polynomial.legendre.leggauss(50) ## used for numerical integration later on
ti = torch.tensor(ti,device = device,dtype=dtype)
wi = torch.tensor(wi,device=device,dtype=dtype)




n_frames = home_pos.shape[1]
first_frame = 0
batch_size = 250

# time to intercept empty torch
tti = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# 28 players*500 batches*grid
tmp2 = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y,1],device = device,dtype=dtype)
# 28 players*500 batches*grid * 1
pc = torch.empty([n_frames,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# frames * grid

# for f in range(int(n_frames/batch_size)):
# taking 500 frames or the last frames if less than 500 left
for f in range(1): 
    
    bp = torch.tensor(ball_pos[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hp = torch.tensor(home_pos[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hv = torch.tensor(home_v[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    ap = torch.tensor(away_pos[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    av = torch.tensor(away_v[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    
    ball_travel_time = torch.norm(target_position - bp, dim=4).div_(average_ball_speed) 
    # ball travel time to each location in each frame in the batch
    
    r_reaction_home = hp + hv.mul_(reaction_time) # position after reaction time (vector)
    r_reaction_away = ap + av.mul_(reaction_time) # = position + velocity multiplied by reaction time
    r_reaction_home = r_reaction_home - target_position # distance to target position (vector)
    r_reaction_away = r_reaction_away - target_position # after reaction time
    
    # time to intercept for home and away filled 
    # torch.norm --> distance of position in grid to target position from vector to value
    # add reaction time to time of intercept
    # divide distance by speed --> time of intercept
    # i think it should be the other way around! Divide first and then add reaction time
    tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).add_(reaction_time).div_(max_player_speed)
    tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).add_(reaction_time).div_(max_player_speed)
    # tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).div_(max_player_speed).add_(reaction_time)
    # tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).div_(max_player_speed).add_(reaction_time)

    tmp2[...,0] = sigma * (ball_travel_time - tti)
    
    tmp1 = sigma * 0.5 * (ti + 1) * 10 + tmp2
    
    
    hh = torch.sigmoid(tmp1[:14]).mul_(4.3)
    h = hh.sum(0)
    
    S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))

    pc[(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,int(first_frame + n_frames)))] = torch.matmul(S*h,wi).mul_(5.)


In [168]:
tmp1.shape

torch.Size([28, 500, 50, 30, 50])

In [167]:
tmp2.shape

torch.Size([28, 500, 50, 30, 1])

In [112]:
print(S.shape)
S

torch.Size([500, 50, 30, 50])


tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [1.0000, 1.0000, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [1.0000, 1.0000, 0.9999,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0

In [113]:
print(h.shape)
h

torch.Size([500, 50, 30, 50])


tensor([[[[6.5866e-05, 7.2606e-05, 8.6482e-05,  ..., 4.6718e+01,
           4.6806e+01, 4.6850e+01],
          [3.1806e-04, 3.5060e-04, 4.1760e-04,  ..., 4.7039e+01,
           4.7080e+01, 4.7100e+01],
          [1.5996e-03, 1.7632e-03, 2.1000e-03,  ..., 4.7173e+01,
           4.7193e+01, 4.7203e+01],
          ...,
          [1.2236e-03, 1.3488e-03, 1.6064e-03,  ..., 4.5708e+01,
           4.5930e+01, 4.6042e+01],
          [2.6633e-04, 2.9358e-04, 3.4968e-04,  ..., 4.4241e+01,
           4.4600e+01, 4.4788e+01],
          [5.8939e-05, 6.4970e-05, 7.7387e-05,  ..., 4.2210e+01,
           4.2663e+01, 4.2908e+01]],

         [[4.4805e-05, 4.9390e-05, 5.8830e-05,  ..., 4.7019e+01,
           4.7063e+01, 4.7084e+01],
          [1.9282e-04, 2.1255e-04, 2.5317e-04,  ..., 4.7186e+01,
           4.7204e+01, 4.7213e+01],
          [9.2107e-04, 1.0153e-03, 1.2093e-03,  ..., 4.7249e+01,
           4.7257e+01, 4.7261e+01],
          ...,
          [7.3568e-04, 8.1096e-04, 9.6591e-04,  ..., 4.6541

In [115]:
S*h

tensor([[[[6.5866e-05, 7.2606e-05, 8.6482e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.1806e-04, 3.5060e-04, 4.1759e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.5996e-03, 1.7631e-03, 2.0998e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [1.2236e-03, 1.3487e-03, 1.6063e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.6633e-04, 2.9358e-04, 3.4967e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [5.8939e-05, 6.4970e-05, 7.7386e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[4.4805e-05, 4.9390e-05, 5.8829e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.9282e-04, 2.1255e-04, 2.5316e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.2107e-04, 1.0153e-03, 1.2092e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [7.3568e-04, 8.1094e-04, 9.6585e-04,  ..., 0.0000

In [120]:
print(wi.shape)
print(wi.sum())
wi

torch.Size([50])
tensor(2.0000)


tensor([0.0029, 0.0068, 0.0106, 0.0144, 0.0181, 0.0218, 0.0254, 0.0288, 0.0322,
        0.0355, 0.0386, 0.0415, 0.0443, 0.0470, 0.0494, 0.0517, 0.0537, 0.0556,
        0.0572, 0.0586, 0.0598, 0.0607, 0.0615, 0.0619, 0.0622, 0.0622, 0.0619,
        0.0615, 0.0607, 0.0598, 0.0586, 0.0572, 0.0556, 0.0537, 0.0517, 0.0494,
        0.0470, 0.0443, 0.0415, 0.0386, 0.0355, 0.0322, 0.0288, 0.0254, 0.0218,
        0.0181, 0.0144, 0.0106, 0.0068, 0.0029])

In [118]:
print(torch.matmul(S*h,wi).shape)
torch.matmul(S*h,wi)
# numerische Integralbildung durch Gewichte (wi) durch Matrixvektorprukt aus Sh und den Gewichten 
# i.e. Gewicht wird mit dem Entsprechenden der 50 Werte aus der Verteilung multipliziert

torch.Size([500, 50, 30])


tensor([[[1.9991e-01, 1.9996e-01, 1.9999e-01,  ..., 2.0000e-01,
          2.0000e-01, 1.9999e-01],
         [1.9973e-01, 1.9988e-01, 1.9995e-01,  ..., 2.0000e-01,
          1.9999e-01, 1.9998e-01],
         [1.9930e-01, 1.9963e-01, 1.9982e-01,  ..., 1.9998e-01,
          1.9997e-01, 1.9993e-01],
         ...,
         [4.5233e-07, 2.0718e-07, 8.8808e-08,  ..., 2.2765e-09,
          2.5070e-09, 2.5907e-09],
         [1.6905e-07, 7.5695e-08, 3.1979e-08,  ..., 1.4090e-09,
          1.7501e-09, 1.9938e-09],
         [6.7047e-08, 2.9749e-08, 1.2552e-08,  ..., 8.1987e-10,
          1.1246e-09, 1.4112e-09]],

        [[1.9991e-01, 1.9996e-01, 1.9999e-01,  ..., 2.0000e-01,
          2.0000e-01, 1.9999e-01],
         [1.9973e-01, 1.9988e-01, 1.9995e-01,  ..., 2.0000e-01,
          1.9999e-01, 1.9998e-01],
         [1.9931e-01, 1.9963e-01, 1.9982e-01,  ..., 1.9999e-01,
          1.9997e-01, 1.9993e-01],
         ...,
         [5.1602e-07, 2.3926e-07, 1.0393e-07,  ..., 2.4454e-09,
          2.645

In [68]:
0.5 * (ti + 1) * 10 # --> tensor of values roughly 0-10
sigma * 0.5 * (ti + 1) * 10 # tensor of roughly 0.02 - 40.02
tmp1 = sigma * 0.5 * (ti + 1) * 10 + tmp2 # the distribution added to the sigma scaled difference between tti & ball travel time = distribution of times to control?!
hh = torch.sigmoid(tmp1[:14]).mul_(4.3)    # same distribution but sigmoid * 4.3 (lambda) --> [0, 4.3], but that makes no sense should be 0-1 based on lambda
# only for one team
h = hh.sum(0) # sum for the entire team 

S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))
#  S looks equivalent to the control probability function divided by sigma (4)
# this would imply that the delta of the softpluses of tmp1 and 2 is the time interval available for control








In [74]:
softplus(tmp1)

tensor([[[[[1.4659e-05, 1.6159e-05, 1.9247e-05,  ..., 2.8858e+01,
            2.9033e+01, 2.9131e+01],
           [7.2957e-05, 8.0423e-05, 9.5792e-05,  ..., 3.0463e+01,
            3.0638e+01, 3.0735e+01],
           [3.7063e-04, 4.0855e-04, 4.8661e-04,  ..., 3.2089e+01,
            3.2263e+01, 3.2361e+01],
           ...,
           [2.8432e-04, 3.1341e-04, 3.7330e-04,  ..., 3.1823e+01,
            3.1998e+01, 3.2096e+01],
           [6.1787e-05, 6.8109e-05, 8.1125e-05,  ..., 3.0297e+01,
            3.0472e+01, 3.0569e+01],
           [1.3628e-05, 1.5022e-05, 1.7893e-05,  ..., 2.8785e+01,
            2.8960e+01, 2.9058e+01]],

          [[8.5095e-06, 9.3803e-06, 1.1173e-05,  ..., 2.8314e+01,
            2.8489e+01, 2.8587e+01],
           [4.1827e-05, 4.6108e-05, 5.4919e-05,  ..., 2.9907e+01,
            3.0082e+01, 3.0179e+01],
           [2.0984e-04, 2.3131e-04, 2.7551e-04,  ..., 3.1520e+01,
            3.1694e+01, 3.1792e+01],
           ...,
           [1.7034e-04, 1.8777e-04, 2.2

In [86]:
A =torch.tensor([0, 1, 2, 3])
B = torch.tensor([-1, 1, 2, 2.5])

In [87]:
torch.sigmoid(A)

tensor([0.5000, 0.7311, 0.8808, 0.9526])

In [88]:
torch.sigmoid(B)

tensor([0.2689, 0.7311, 0.8808, 0.9241])

In [80]:
S.unique()

tensor([0.0000e+00, 1.4013e-45, 2.8026e-45,  ..., 1.0000e+00, 1.0000e+00,
        1.0000e+00])

In [100]:
def lambda_sigmoid(x, lam=4.3):
    return torch.sigmoid(x).mul_(lam)

def cp(x, lam=4.3):
    return(1-torch.exp(-lam*x))

def pt(x, lam=4.3):
    return -lam * torch.exp(-lam*x)

In [94]:
x =  torch.tensor([1, 2, 3, 4])

In [99]:
lambda_sigmoid(x)/4.3

tensor([0.7311, 0.8808, 0.9526, 0.9820])

In [97]:
cp(x)

tensor([0.9864, 0.9998, 1.0000, 1.0000])

In [101]:
pt(x)

tensor([-5.8345e-02, -7.9165e-04, -1.0742e-05, -1.4575e-07])

In [102]:
torch.exp(x)

tensor([ 2.7183,  7.3891, 20.0855, 54.5981])

tensor([ 2.7183,  7.3891, 20.0855, 54.5982], dtype=torch.float64)

In [108]:
pc.shape

torch.Size([145005, 50, 30])

In [110]:
pc

tensor([[[9.9955e-01, 9.9982e-01, 9.9994e-01,  ..., 9.9999e-01,
          9.9999e-01, 9.9997e-01],
         [9.9863e-01, 9.9939e-01, 9.9976e-01,  ..., 9.9998e-01,
          9.9995e-01, 9.9990e-01],
         [9.9652e-01, 9.9814e-01, 9.9911e-01,  ..., 9.9992e-01,
          9.9984e-01, 9.9966e-01],
         ...,
         [2.2616e-06, 1.0359e-06, 4.4404e-07,  ..., 1.1382e-08,
          1.2535e-08, 1.2953e-08],
         [8.4524e-07, 3.7848e-07, 1.5990e-07,  ..., 7.0451e-09,
          8.7507e-09, 9.9689e-09],
         [3.3524e-07, 1.4875e-07, 6.2758e-08,  ..., 4.0993e-09,
          5.6232e-09, 7.0558e-09]],

        [[9.9955e-01, 9.9982e-01, 9.9994e-01,  ..., 9.9999e-01,
          9.9999e-01, 9.9997e-01],
         [9.9863e-01, 9.9939e-01, 9.9976e-01,  ..., 9.9998e-01,
          9.9996e-01, 9.9990e-01],
         [9.9653e-01, 9.9815e-01, 9.9911e-01,  ..., 9.9993e-01,
          9.9984e-01, 9.9966e-01],
         ...,
         [2.5801e-06, 1.1963e-06, 5.1966e-07,  ..., 1.2227e-08,
          1.322

In [169]:
np.pi / np.sqrt(3.) / 0.45

4.030665253853817

In [170]:
 np.pi * 1/ np.sqrt(3.) / 0.45

4.030665253853817

In [171]:
np.sqrt(3.) * 0.45 / np.pi

0.2480980029398064